In [1]:

from pyphm.datasets.ims import ImsDataLoad
from pathlib import Path
import pandas as pd
import os
import numpy as np
import time
import datetime
import csv

%load_ext autoreload
%autoreload 2

In [2]:
root_dir = Path.cwd().parent
print(root_dir)
path_data_raw_folder = Path(root_dir / 'data' )
print(path_data_raw_folder)

/home/tim/Documents/PyPHM
/home/tim/Documents/PyPHM/data


In [9]:
ims = ImsDataLoad(path_data_raw_folder, 'ims', download=False)

path_1st_folder = ims.path_1st_folder
path_2nd_folder = ims.path_2nd_folder
path_3rd_folder = ims.path_3rd_folder

file_list = sorted(os.listdir(path_1st_folder))
file_name = file_list[0]

type(root) =  <class 'pathlib.PosixPath'>
/home/tim/Documents/PyPHM/data/ims


In [18]:
def process_raw_csv_ims(path_run_folder, file_name, sample_index, run_no, sample_freq=20480.0):

    signals = np.loadtxt(path_run_folder / file_name, delimiter="\t")

    id_list = [f'{run_no}_{sample_index}'] * len(signals)
    run = [run_no] * len(signals)
    file = [file_name] * len(signals)
    time_step = np.linspace(0.0, len(signals) / sample_freq, len(signals))

    return signals, id_list, run, file, time_step

In [42]:
file_list = sorted(os.listdir(path_1st_folder))
file_name = file_list[0]

signals, id_list, run, file, time_step = process_raw_csv_ims(path_1st_folder, file_name, 0, 1, sample_freq=20480.0)

In [43]:
len(id_list)

20480

In [44]:
run.extend(run)


In [45]:
len(run)

40960

In [46]:
a = [signals, signals]
a = np.concatenate(a, axis=0)
a.shape

(40960, 8)

In [47]:
df = pd.DataFrame(a, columns=ims.col_1st_names)
df['run'] = run
df.tail()

,b1_ch1,b1_ch2,b2_ch3,b2_ch4,b3_ch5,b3_ch6,b4_ch7,b4_ch8,run
40955,-0.022,-0.142,0.085,-0.078,-0.200,-0.198,-0.134,-0.093,1
40956,0.024,-0.156,0.042,-0.039,-0.159,-0.071,-0.227,-0.132,1
40957,-0.193,-0.303,-0.071,-0.125,-0.237,-0.251,-0.151,0.007,1
40958,-0.115,0.076,-0.078,-0.332,-0.027,-0.002,-0.151,-0.005,1
40959,-0.042,0.098,0.103,-0.059,-0.002,-0.342,-0.095,-0.276,1


In [16]:
run_no = 1
sample_index = 0
id_list = [f'{run_no}_{sample_index}']*10

id_list

['1_0', '1_0', '1_0', '1_0', '1_0', '1_0', '1_0', '1_0', '1_0', '1_0']

In [11]:
a = np.loadtxt(path_1st_folder / file_name, delimiter="\t")

col_names = ['b1_ch1', 'b1_ch2', 'b2_ch3', 'b2_ch4', 'b3_ch5', 'b3_ch6', 'b4_ch7', 'b4_ch8']
df = pd.read_csv(path_1st_folder / file_name, sep='\t', names=col_names)
df.head()

,b1_ch1,b1_ch2,b2_ch3,b2_ch4,b3_ch5,b3_ch6,b4_ch7,b4_ch8
0,-0.022,-0.039,-0.183,-0.054,-0.105,-0.134,-0.129,-0.142
1,-0.105,-0.017,-0.164,-0.183,-0.049,0.029,-0.115,-0.122
2,-0.183,-0.098,-0.195,-0.125,-0.005,-0.007,-0.171,-0.071
3,-0.178,-0.161,-0.159,-0.178,-0.100,-0.115,-0.112,-0.078
4,-0.208,-0.129,-0.261,-0.098,-0.151,-0.205,-0.063,-0.066


In [13]:
a[1]

array([-0.105, -0.017, -0.164, -0.183, -0.049,  0.029, -0.115, -0.122])

In [15]:
df = ims.load_run_as_dataframe(run_no=1)

len file_list: 2156
Processing 2003.10.22.12.06.24, i=0 - run 1
Processing 2003.10.22.16.54.13, i=50 - run 1
Processing 2003.10.23.01.04.13, i=100 - run 1
Processing 2003.10.23.09.24.13, i=150 - run 1
Processing 2003.10.29.21.59.46, i=200 - run 1
Processing 2003.10.31.02.39.46, i=250 - run 1
Processing 2003.10.31.12.41.44, i=300 - run 1
Processing 2003.10.31.21.01.44, i=350 - run 1
Processing 2003.11.01.05.21.44, i=400 - run 1
Processing 2003.11.01.13.41.44, i=450 - run 1
Processing 2003.11.01.22.01.44, i=500 - run 1
Processing 2003.11.07.16.31.44, i=550 - run 1
Processing 2003.11.08.12.21.44, i=600 - run 1
Processing 2003.11.08.20.41.44, i=650 - run 1
Processing 2003.11.09.05.01.44, i=700 - run 1
Processing 2003.11.09.13.15.58, i=750 - run 1
Processing 2003.11.10.01.05.58, i=800 - run 1
Processing 2003.11.10.09.25.58, i=850 - run 1
Processing 2003.11.14.12.12.17, i=900 - run 1
Processing 2003.11.14.20.58.46, i=950 - run 1
Processing 2003.11.15.05.18.46, i=1000 - run 1
Processing 2003.

In [16]:
df.head()

,b1_ch1,b1_ch2,b2_ch3,b2_ch4,b3_ch5,b3_ch6,b4_ch7,b4_ch8
0,-0.022,-0.039,-0.183,-0.054,-0.105,-0.134,-0.129,-0.142
1,-0.105,-0.017,-0.164,-0.183,-0.049,0.029,-0.115,-0.122
2,-0.183,-0.098,-0.195,-0.125,-0.005,-0.007,-0.171,-0.071
3,-0.178,-0.161,-0.159,-0.178,-0.100,-0.115,-0.112,-0.078
4,-0.208,-0.129,-0.261,-0.098,-0.151,-0.205,-0.063,-0.066


In [17]:
df.shape

(44154880, 8)

In [4]:
path_1st_folder = ims.path_1st_folder
path_2nd_folder = ims.path_2nd_folder
path_3rd_folder = ims.path_3rd_folder

In [5]:
file_list = sorted(os.listdir(path_3rd_folder))
file_list[0]

'2004.03.04.09.27.46'

In [6]:
file_name = file_list[0]
print(file_name)

col_names = ['b1_ch1', 'b1_ch2', 'b2_ch3', 'b2_ch4', 'b3_ch5', 'b3_ch6', 'b4_ch7', 'b4_ch8']
df = pd.read_csv(path_3rd_folder / file_name, sep='\t', names=col_names)
df.head()

2004.03.04.09.27.46


,b1_ch1,b1_ch2,b2_ch3,b2_ch4,b3_ch5,b3_ch6,b4_ch7,b4_ch8
0,0.034,0.264,0.039,-0.046,NaN,NaN,NaN,NaN
1,0.103,0.083,-0.061,-0.012,NaN,NaN,NaN,NaN
2,0.095,-0.039,-0.007,0.039,NaN,NaN,NaN,NaN
3,0.000,0.110,0.022,-0.002,NaN,NaN,NaN,NaN
4,0.005,0.154,-0.127,-0.020,NaN,NaN,NaN,NaN


In [7]:
# columns to add
# ['id', 'run', 'start_time_unix', 'date_time', 'unix_time', 'file_name']

df['id'] = "1_" + file_name
df['run'] = 1
df.head()

,b1_ch1,b1_ch2,b2_ch3,b2_ch4,b3_ch5,b3_ch6,b4_ch7,b4_ch8,id,run
0,0.034,0.264,0.039,-0.046,NaN,NaN,NaN,NaN,1_2004.03.04.09.27.46,1
1,0.103,0.083,-0.061,-0.012,NaN,NaN,NaN,NaN,1_2004.03.04.09.27.46,1
2,0.095,-0.039,-0.007,0.039,NaN,NaN,NaN,NaN,1_2004.03.04.09.27.46,1
3,0.000,0.110,0.022,-0.002,NaN,NaN,NaN,NaN,1_2004.03.04.09.27.46,1
4,0.005,0.154,-0.127,-0.020,NaN,NaN,NaN,NaN,1_2004.03.04.09.27.46,1


In [8]:
def process_raw_csv_ims(path_run_folder, file_name, sample_index, run_no, col_names, sample_freq=20480.0):

    df = pd.read_csv(path_run_folder / file_name, sep='\t', names=col_names)

    df['id'] = "1_" + str(sample_index)
    df['run'] = run_no
    df['file'] = file_name
    df['time'] = np.linspace(0.0, len(df)/sample_freq, len(df))

    return df[['id', 'file', 'run', 'time',] + col_names]

In [27]:
df1 = process_raw_csv_ims(path_3rd_folder, file_name, 0, 3, col_names, sample_freq=20480.0)
df1.head()

,id,file,run,time,b1_ch1,b1_ch2,b2_ch3,b2_ch4,b3_ch5,b3_ch6,b4_ch7,b4_ch8
0,1_0,2004.03.04.09.27.46,3,0.000000,0.034,0.264,0.039,-0.046,NaN,NaN,NaN,NaN
1,1_0,2004.03.04.09.27.46,3,0.000049,0.103,0.083,-0.061,-0.012,NaN,NaN,NaN,NaN
2,1_0,2004.03.04.09.27.46,3,0.000098,0.095,-0.039,-0.007,0.039,NaN,NaN,NaN,NaN
3,1_0,2004.03.04.09.27.46,3,0.000146,0.000,0.110,0.022,-0.002,NaN,NaN,NaN,NaN
4,1_0,2004.03.04.09.27.46,3,0.000195,0.005,0.154,-0.127,-0.020,NaN,NaN,NaN,NaN


In [ ]:
def create_xy_dataframe():

In [29]:
df1['time'][2]

9.766101860442404e-05

In [8]:
# convert start_time to unix timestamp
start_time_unix = time.mktime(
    datetime.datetime.strptime(file_name, "%Y.%m.%d.%H.%M.%S").timetuple()
)
start_time

1066849584.0

In [9]:
date_nice_format = datetime.datetime.fromtimestamp(start_time).strftime(
    "%Y-%m-%d %H:%M:%S"
)  # reformat date
date_nice_format


'2003-10-22 12:06:24'

In [7]:
date_list = sorted(os.listdir(path_1st_folder))

In [10]:
date_list[:10]

['2003.10.22.12.06.24',
 '2003.10.22.12.09.13',
 '2003.10.22.12.14.13',
 '2003.10.22.12.19.13',
 '2003.10.22.12.24.13',
 '2003.10.22.12.29.13',
 '2003.10.22.12.34.13',
 '2003.10.22.12.39.13',
 '2003.10.22.12.44.13',
 '2003.10.22.12.49.13']